# Social Network Analysis Project

## Table Of Content

##### 1. [Introduction](#intro-header)

1. [The Data](#data)
2. [Research Question](#research-question)
3. [Our Approach](#approach)

##### 2. [Building the Network](#build-header)

1. [Data Cleansing](#cleansing)
2. [Create the Network](#create)
3. [Correlation between features](#feat-matrix)

##### 3. [Distributions](#dis-header)

1. [Degree Distributions](#degree-dis)
2. [Betweenness Distributions](#bet-dis)


<hr>

# 1. Introduction <a class="anchor" id="intro-header"></a>
## 1.1. The Data <a class="anchor" id="data"></a>

We chose the Cora Citation Network, it is a directed network where nodes represent scientific papers, an edge between two nodes indicates that the left node cites the right node, the edges are unweighted. In addition the papers are classified to categories and sub-categories.

## 1.2. Research Question <a class="anchor" id="research-question"></a>

1. What is the most cited sub-category that all other categories depend on?
2. If we cluster our nodes (look for communities within the network) what catergies will be in each community?

We expect for each community to include articles from the same category or sub-category depending on number of communities we look for.  


## 1.3. Approach for our research <a class="anchor" id="approach"></a>

##### Degree Centrality
We'll calculate two In Degrees for each node:
1. The in degree of the node among it's category.
2. The in degree of the node according to all categories besides the node's category. 

Both of the degrees above will be normalized so we can compare them. We will use the second measure in order to provide an answer to our first research question.


##### Community Detection
We want to check if the clusters are of nodes from the same category or sub-catergoy, by using two methods: 
1. Removing edges with high betweenness and checking the resulted communities - in other words The Girvan-Newman Method.
2. Using either the Modularity method or Louvain community detection method.  
<b>#TODO change to selected one above</b>


### Imports

In [1]:
# Import packages
import numpy as np
import os
import pandas as pd
import networkx as nx
import time
from random import sample
import sys
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go

<hr>

# 2. Building the Network <a class="anchor" id="build-header"></a>
## 2.1. Data Cleansing <a class="anchor" id="cleansing"></a>

Get data from csv's:

In [32]:
# edges.csv has 91500 rows, nodes.csv has 23166 rows
edges = pd.read_csv('./data/edges.csv', delimiter=' ')
nodes = pd.read_csv('./data/nodes.csv', delimiter=' ')
nodes = nodes[['network_id', 'node_id']]

#get id's, get each node id's category and concat them together in names df
names = pd.read_csv('./data/ent.subelj_cora_cora.id.csv', delimiter=' ')
cat = pd.read_csv('./data/ent.subelj_cora_cora.class.csv', delimiter=' ')
names['category'] = cat[['category']]

result = pd.merge(nodes, names, on='node_id')
result = result.drop(['node_id'], axis=1)

# split category and sub-category
cat_df = result.copy()
cat_df['category'] = cat_df['category'].astype(str)

for index, row in cat_df.iterrows():
    split = cat_df.loc[index]['category'].split('/')
    cat = split[1]
    cat = cat.replace('_', ' ')
    cat_df.loc[index, 'category'] = cat
    subcat = split[2]
    subcat = subcat.replace('_', ' ')
    cat_df.loc[index, 'sub_cat'] = subcat

Final data contains two dataframes one for the edges and the other includes each node and its category and sub-category, an example:

In [35]:
edges.head(2)

,cites,cited
0,20128,6078
1,22236,10436


In [34]:
cat_df.head(2)

,network_id,category,sub_cat
0,1,Databases,Performance
1,2,Human Computer Interaction,Cooperative


In [8]:
'''
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
# Encode labels in column 'species'. 
cat_df['category']= label_encoder.fit_transform(cat_df['category']) 
label_encoder.fit_transform(cat_df['category']) 
'''

array([2, 5, 9, ..., 0, 4, 0], dtype=int64)

## 2.2. Create the Network <a class="anchor" id="create"></a>


In [36]:
edges_tuple = [tuple(x) for x in edges.to_numpy()]
DG = nx.DiGraph()
DG.add_edges_from(edges_tuple)

Some of the network's properties:

In [37]:
print(nx.info(DG))
print("Is the Graph directed? " + str(DG.is_directed()))
print("Graph Density is: " + str(nx.density(DG)))
print("Average Clustering: " + str(nx.average_clustering(DG, nodes=None, weight=None, count_zeros=True)))

Name: 
Type: DiGraph
Number of nodes: 23166
Number of edges: 91500
Average in degree:   3.9498
Average out degree:   3.9498
Is the Graph directed? True
Graph Density is: 0.00017050524281260305
Average Clustering: 0.14601503382808564


<hr>

# 3. Distributions <a class="anchor" id="dis-header"></a>
## 3.1. Degree Distribution <a class="anchor" id="degree-dis"></a>

Since our network is directed we'll calculate both in and out degree for each node, we expect to get power-law distribution in both.

In [66]:
'''
Given the "list" of degrees we get from xnetwork this method turns it into a 
df with two columns one for degree and other count
'''
def degree_to_df(degree):
    all_degrees = []
    for node,deg in list(degree):
        all_degrees.append(deg)
    unique_degrees = list(set(all_degrees))

    count = []
    for i in unique_degrees:
        x = all_degrees.count(i)
        count.append(x)

    df = pd.DataFrame(list(zip(unique_degrees, count)), 
                   columns =['Degree', 'Freq']) 
    return df 

In [67]:
out_deg = DG.out_degree()
in_deg = DG.in_degree()
out_df = degree_to_df(out_deg)
in_df = degree_to_df(in_deg)

In [84]:
# Plot the in and out degree distribution
fig =  plotly.subplots.make_subplots(rows=1, cols=2, horizontal_spacing=0.1,
                                     subplot_titles=("In-Degree Distribution","Out-Degree Distribution"),
                                                     specs=[[{"type": "xy"},{"type": "xy"}]])
fig.add_trace(
    go.Scatter(x=in_df['Degree'], y=in_df['Freq'],marker_symbol='hexagon2', mode="markers+text", 
               marker=dict(size=12,color='rgba(135, 206, 250, 0.7)', line=dict(width=1, color='DarkSlateGrey'))), row=1, col=1)
fig.add_trace(
go.Scatter(x=out_df['Degree'], y=out_df['Freq'],marker_symbol='hexagon2', mode="markers+text", 
               marker=dict(size=12,color='rgba(135, 206, 250, 0.7)', line=dict(width=1, color='DarkSlateGrey'))), row=1, col=2)
fig.update_xaxes(title_text="Degree")
fig.update_yaxes(title_text="Frequency")
fig.update_layout(height=500, width=1000,showlegend=False)
fig.show()

#### Results 

As expected both the in and out degree distributions are the power-law distribution which means small occurences are extremely common, notice that in the number of in-degree of 0 is much higher than in out-degree which makes sence since most articles cite at least another article but there is a big number of articles that no one cited so far. 


## 3.2. Betweenness Distribution <a class="anchor" id="bet-dis"></a>
